### Collisional-radiative model for Argon Plasma
#### Rates reading
See: https://doi.org/10.1063/1.3585688

In [1]:
import os
import json
import numpy as np
import dill as pickle

Inputs

In [2]:
path_to_species = "./../species"
path_to_dtb = "/home/zanardi/Codes/ML/RONEK/run/Argon/argon_cr/data"
T_min = 200

In [3]:
# Save path
out_path = "./../rates/"
os.makedirs(out_path, exist_ok=True)

Read species

In [4]:
species = {}
for filename in os.listdir(path_to_species):
  filename = path_to_species + "/" + filename
  with open(filename, "r") as file:
    ispecies = json.load(file)
  species[ispecies["name"]] = ispecies

Read temperature grid

In [5]:
T_full = np.loadtxt(path_to_dtb+"/T", skiprows=4)
indices = np.where(T_full>T_min)[0]
T = T_full[indices]

Dimensions

In [6]:
nb_pts = len(indices)
nb_comp_Ar = species["Ar"]["nb_comp"]
nb_comp_Arp = species["Arp"]["nb_comp"]

**Kinetics**

In [7]:
kin = {"T": T}

- Excitation by electron impact (`EXe`):
$$\quad \text{Ar}(i) + \text{e}^- \xrightleftharpoons[F_{ji}]{C_{ij}} \text{Ar}(j) + \text{e}^-; \quad j > i$$
- Excitation by heavy-particle impact (`EXh`):
$$\quad \text{Ar}(i) + \text{Ar}(1) \xrightleftharpoons[L_{ji}]{K_{ij}} \text{Ar}(j) + \text{Ar}(1); \quad j > i$$

In [8]:
def read_exc(name):
  X = np.zeros((nb_comp_Ar,1,nb_comp_Ar,1,nb_pts))
  for i in range(nb_comp_Ar):
    for j in range(i+1,nb_comp_Ar):
      X[i,0,j,0] = np.loadtxt(path_to_dtb+f"/{name}{i}_{j}")[indices]
  return X

In [9]:
kin["EXe"] = {
  "plato_id": 7,
  "values": read_exc("C"),
  "equation": "Ar(*)+em=Ar(*)+em"
}
kin["EXh"] = {
  "plato_id": 6,
  "values": read_exc("K"),
  "equation": "Ar(*)+Ar(0)=Ar(*)+Ar(0)"
}
# Correction factor applied to the rate from the ground state (i=0)
# See: M. G. Kapper's PhD thesis (2009)
kin["EXh"]["values"][0,0,:,0] *= 0.085

- Ionization by electron impact (`Ie`)
$$\quad \text{Ar}(i) + \text{e}^- \xrightleftharpoons[O_{ji}]{S_{ij}} \text{Ar}^+(j) + \text{e}^- + \text{e}^-$$
- Ionization by heavy-particle impact (`Ih`)
$$\quad \text{Ar}(i) + \text{Ar}(1) \xrightleftharpoons[W_{ji}]{V_{ij}} \text{Ar}^+(j) + \text{e}^- + \text{Ar}(1)$$

In [10]:
def read_ion(name):
  X = np.zeros((nb_comp_Ar,1,nb_comp_Arp,1,1,nb_pts))
  for i in range(nb_comp_Ar):
    found = False
    jc_i = species["Ar"]["lev"]["jc"][i]
    for j in range(nb_comp_Arp):
      jc_j = species["Arp"]["lev"]["jc"][j]
      # Ar+ and Ar_i must have the same value for the core angular momentum
      if (jc_i == jc_j):
        found = True
        X[i,0,j,0,0] = np.loadtxt(path_to_dtb+f"/{name}{i}")[indices]
        break
    if (not found):
      raise ValueError("Ionization rate not found! Check 'jc' values.")
  return X

In [11]:
kin["Ie"] = {
  "plato_id": 9,
  "values": read_ion("S"),
  "equation": "Ar(*)+em=Arp(*)+em+em"
}
kin["Ih"] = {
  "plato_id": 8,
  "values": read_ion("V"),
  "equation": "Ar(*)+Ar(0)=Arp(*)+Ar(0)+em"
}

- Electron-neutral collision rate (First moment)

In [12]:
kin["EN"] = {
  "values": np.loadtxt(path_to_dtb+"/ken")[indices],
  "equation": None,
  # Curve fit model
  # > See: Capitelli et al., https://doi.org/10.1007/978-1-4419-8172-1, Eq. 11.3
  "Q11_fit": np.array([
    +8.8972699e+0,
    +1.5224377e+0,
    +2.4749525e+0,
    -2.0999602e+0,
    -1.0000000e+0,
    +0.0000000e+0,
    +2.2908023e+0,
    +1.0598130e+1,
    -9.5311431e-1,
    +1.4563107e+1
  ])
}
kin["EI"] = {
  "values": None,
  "equation": None,
  # Curve fit model
  # > See: Magin's PhD thesis, ULB, 2004
  "Q11_fit": np.array([
    -1.67070199e-4,
    +4.87662165e-3,
    -6.34929831e-2,
    +5.49816993e-1,
    -7.91157696e-1
  ])
}

**Radiation**

In [13]:
rad = {"T": T}

Radiation absorption is taken into account via the use of escape $\Lambda_{ij}$ factors that can take values between 0 (optically thick plasma) and 1 (optically thin plasma). An optically thin plasma is considered here ($\Lambda_{ij}=1$).
- Spontaneous emission (bound-bound): $$\quad \text{Ar}(i) + h\nu_{ij} \xrightleftharpoons[\Lambda_{ij}A_{ij}]{(1-\Lambda_{ij})A_{ij}} \text{Ar}(j)$$

In [14]:
rad["BB"] =  {
  "values": np.loadtxt(path_to_dtb+"/A"),
  "equation": "Ar(*)=Ar(*)"
}

- Photo-ionization (bound-free and free-bound):
$$\quad \text{Ar}(i) + h\nu_{ij} \xrightleftharpoons[\Lambda_{ij}R_{ij}]{(1-\Lambda_{ij})R_{ij}} \text{Ar}^+(j) + \text{e}^-$$

In [15]:
rad["PI"] =  {
  "plato_id": 11,
  "values": read_ion("R"),
  "equation": "Ar(*)=Arp(*)+em"
}
# First moment
rad["PIp"] =  {
  "plato_id": 11,
  "values": read_ion("Rp"),
  "equation": "Ar(*)=Arp(*)+em"
}

Save raw rates

In [16]:
pickle.dump(kin, open(out_path + "/kin_raw.p", "wb"))
pickle.dump(rad, open(out_path + "/rad_raw.p", "wb"))